Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=one-bookshelf%2Fbookmark-no-hp-rpg-370430%2Fbookmark_no_hp_rpg.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/one-bookshelf/bookmark-no-hp-rpg-370430)]*

## [``dyce``](https://posita.github.io/dyce/) computation of mechanic odds for [“Bookmark No HP RPG”](https://www.drivethrurpg.com/product/370430/Bookmark-No-HP-RPG)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

### Interpreting the graphs below

A zero (0) indicates the likelihood of failure.
Other values indicate the likelhood of that outcome as the maximum value from all rolls.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.4", "dyce~=0.6.2"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H
from functools import reduce
from itertools import repeat

dice = {"d4": H(4), "d6": H(6), "d8": H(8), "d10": H(10), "d12": H(12)}

def bookmark_rpg(n: int, d: H) -> H:
    lowest_outcome = min(d)
    bool_lowest_does_not_appear = (n @ d.eq(lowest_outcome)).eq(0)
    d_without_lowest = d.draw(lowest_outcome)
    max_success = reduce(
        lambda lh, rh: lh.map(max, rh),
        repeat(d_without_lowest, n),
    )
    return (bool_lowest_does_not_appear * max_success).lowest_terms()

In [3]:
from anydyce import jupyter_visualize

jupyter_visualize(
    [
        (f"{d_str} max outcome\nvs. difficulty of {difficulty}", bookmark_rpg(difficulty, dice[d_str]))
        for difficulty in range(1, 11) for d_str in dice
    ],
    controls_expanded=True,
    initial_burst_color_bg_trnsp=True,
    initial_burst_columns=len(dice),
    initial_enable_cutoff=False,
    initial_resolution=18,
)